In [2]:
import os, sys
HOME = os.environ['HOME']  # change if necessary
sys.path.append(f'{HOME}/Finite-groups/src')

In [3]:
import torch as t
import numpy as np
from matplotlib import pyplot as plt
import json
from itertools import product
from jaxtyping import Float
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import copy
import math
from itertools import product
import pandas as pd
from typing import Union
from einops import repeat
from huggingface_hub import snapshot_download
from huggingface_hub.utils import disable_progress_bars


from model import MLP3, MLP4, InstancedModule
from utils import *
from group_data import *
from model_utils import *
from group_utils import *
%load_ext autoreload
%autoreload 2

Syntax warning: Unbound global variable in /usr/share/gap/pkg/browse/PackageIn\
fo.g:73
  if not IsKernelExtensionAvailable("Browse", "ncurses") then
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/edim/PackageInfo\
.g:60
  if not IsKernelExtensionAvailable("EDIM","ediv") then
         ^^^^^^^^^^^^^^^^^^^^^^^^^^


In [70]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")
# MODEL_DIR = '2024-08-16_22-53-29_SL2_5_MLP3_256'
# MODEL_DIR = '2024-08-14_21-24-30_gapS_5_'
# MODEL_DIR = '2024-08-27_16-32-46_Dic30_MLP3_128'
# MODEL_DIR = '2024-08-27_23-16-05_F11_MLP3_256'
MODEL_DIR = '2024-09-02_18-17-20_smallgrp_54_8_MLP3_256_0_6'
disable_progress_bars()
local_dir = f'{HOME}/models/{MODEL_DIR}'
if not os.path.exists(local_dir):
    snapshot_download(repo_id=f'wiwu2390/{MODEL_DIR}', local_dir=local_dir)
models, params = load_models(local_dir)
data = GroupData(params)
group = data.groups[0]

Intersection size: 2916/2916 (1.00)
Added 2916 elements from intersection
Added 0 elements from group 0: smallgrp(54,8)
Taking random subset: 1749/2916 (0.60)
Train set size: 1749/2916 (0.60)


/home/wilson/Finite-groups/src/model_utils.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(t.load(model_path, map_location=device))


In [71]:
loss_dict = test_loss(models[-1].to(device), data)
print(loss_dict['G0_loss'].min(), loss_dict['G0_loss'].argmin())
instance = loss_dict['G0_loss'].argmin().item()
model = models[-1][instance].to(device)

tensor(0.0173) tensor(94)


In [72]:
# model.get_neurons() folds the linear map into the embeddings
lneurons, rneurons = model.get_neurons()
uneurons = model.unembedding.data.detach()
lneurons, rneurons, uneurons = lneurons.squeeze(0).to('cpu'), rneurons.squeeze(0).to('cpu'), uneurons.squeeze(0).to('cpu').T
lneurons.shape, rneurons.shape, uneurons.shape

(torch.Size([54, 256]), torch.Size([54, 256]), torch.Size([54, 256]))

In [73]:
irreps = group.get_real_irreps(verbose=True)
# irreps = group.get_complex_irreps()
print(irreps.keys())

Complex irrep 3d-0 -> real irrep 6d-0
Complex irrep 3d-1 -> real irrep 6d-1
Complex irrep 3d-2 -> real irrep 6d-2
Complex irrep 3d-3 -> real irrep 6d-3
dict_keys(['1d-0', '1d-1', '2d-0', '2d-1', '2d-2', '2d-3', '6d-0', '6d-1', '6d-2', '6d-3'])


In [74]:
# Get orthogonal basis for each irrep
irrep_bases = dict()
for name, irrep in irreps.items():
    irrep = einops.rearrange(irrep, 'N d1 d2 -> N (d1 d2)')
    U, S, V = t.svd(irrep)
    nonzero = S > 1e-5
    irrep_bases[name] = U[:,nonzero]

# Dimension of subspace spanned by each irrep
for k, v in irrep_bases.items():
    print(k, v.shape)

1d-0 torch.Size([54, 1])
1d-1 torch.Size([54, 1])
2d-0 torch.Size([54, 4])
2d-1 torch.Size([54, 4])
2d-2 torch.Size([54, 4])
2d-3 torch.Size([54, 4])
6d-0 torch.Size([54, 18])
6d-1 torch.Size([54, 18])
6d-2 torch.Size([54, 18])
6d-3 torch.Size([54, 18])


In [75]:
# Proportion of variance explained by each irrep, for each neuron
lexpl, rexpl, uexpl = dict(), dict(), dict()

for name, basis in irrep_bases.items():
    lproj = basis.H @ lneurons
    lexpl[name] = (basis.H @ lneurons).norm(dim=0)**2 / (lneurons.norm(dim=0)**2 + 1e-8)
    rexpl[name] = (basis.H @ rneurons).norm(dim=0)**2 / (rneurons.norm(dim=0)**2 + 1e-8)
    uexpl[name] = (basis.H @ uneurons).norm(dim=0)**2 / (uneurons.norm(dim=0)**2 + 1e-8)

lexpl_max, lirrep = t.stack(list(lexpl.values()), dim=0).max(dim=0)

In [76]:
# Get Frobenius-Schur indicators
# +1 means real irrep. Otherwise, complex.
for name, irrep in irreps.items():
    print(name, group.get_frobenius_schur(irrep))

1d-0 1
1d-1 1
2d-0 1
2d-1 1
2d-2 1
2d-3 1
6d-0 0
6d-1 0
6d-2 0
6d-3 0


In [77]:

irrep_idx_dict = {
    name: [
        i for i in range(lneurons.shape[-1]) 
        if lexpl[name][i].item() > 0.95 and lneurons[:,i].norm() > 1e-4
    ]
    for name in irreps
}
for k, v in irrep_idx_dict.items():
    print(k, len(v))

1d-0 0
1d-1 2
2d-0 0
2d-1 0
2d-2 11
2d-3 10
6d-0 108
6d-1 0
6d-2 108
6d-3 0


In [79]:
irrep_name = '6d-0'
irrep_d = irreps[irrep_name].shape[-1]
irrep = irreps[irrep_name]
irrep_idxs = irrep_idx_dict[irrep_name]
irrep_lneurons = lneurons[:, irrep_idxs]
irrep_rneurons = rneurons[:, irrep_idxs]
irrep_uneurons = uneurons[:, irrep_idxs]

In [101]:
def lsq(X, y, lam=1e-5):
    if len(X.shape) == 1:
        X = X.unsqueeze(1)
    if len(y.shape) == 1:
        y = y.unsqueeze(1)
    return (t.linalg.inv(X.T @ X + lam * t.eye(X.shape[1])) @ X.T @ y).flatten()

In [81]:
def rank1_optimize(ln, rn, un, iters=100000):
    def obj(a, b, c, coef):
        return sum(
                [
                    (ln - einops.einsum(b, irrep, a, 'd1, n d1 d2, d2 -> n')).norm()**2,
                    (rn - einops.einsum(a, irrep, c, 'd1, n d1 d2, d2 -> n')).norm()**2,
                    (un - coef * einops.einsum(b, irrep, c, 'd1, n d1 d2, d2 -> n')).norm()**2,
                ]
            )
    
    a = t.randn(irrep_d)
    b = t.randn(irrep_d)
    c = t.randn(irrep_d)
    coef = t.randn(1)

    for j in tqdm(range(100000)):
        # minimize wrt b
        irrep_a = einops.einsum(irrep, a, 'n d1 d2, d2 -> n d1')
        irrep_c = einops.einsum(irrep, c, 'n d1 d2, d2 -> n d1')
        b = lsq(t.concat([irrep_a, coef * irrep_c], dim=0), t.concat([ln, un], dim=0))
        b = b.flatten()
        # minimize wrt c
        a_irrep = einops.einsum(a, irrep, 'd1, n d1 d2 -> n d2')
        b_irrep = einops.einsum(b, irrep, 'd1, n d1 d2 -> n d2')
        c = lsq(t.concat([a_irrep, coef * b_irrep], dim=0), t.concat([rn, un], dim=0))
        # minimize wrt coef
        b_irrep_c = einops.einsum(b, irrep, c, 'd1, n d1 d2, d2 -> n')
        coef = lsq(b_irrep_c, un)
        # minimize wrt a
        b_irrep = einops.einsum(b, irrep, 'd1, n d1 d2 -> n d2')
        irrep_c = einops.einsum(irrep, c, 'n d1 d2, d2 -> n d1')
        a = lsq(t.concat([b_irrep, irrep_c], dim=0), t.concat([ln, rn], dim=0))
        if j % 10000 == 0:
            print(obj(a, b, c, coef))

    return a, b, c, coef

In [104]:
def full_rank_optimize(ln, rn, un, iters=100000):
    def obj(A, B, coef):
        terms = [
            (ln - einops.einsum(irrep, A.T,  'n d1 d2, d1 d2 -> n')).norm()**2 / ln.norm()**2,
            (rn - einops.einsum(irrep, B.T, 'n d1 d2, d1 d2 -> n')).norm()**2 / rn.norm()**2,
            (un - coef * einops.einsum(irrep, (B @ A).T, 'n d1 d2, d1 d2 -> n')).norm()**2 / un.norm()**2,
        ]
        print(terms)
        return sum(terms)

    A = t.randn(irrep_d, irrep_d)
    B = t.randn(irrep_d, irrep_d)
    coef = t.randn(1)

    A = lsq(irrep.flatten(1, 2), ln).reshape((irrep_d, irrep_d)).T
    B = lsq(irrep.flatten(1, 2), rn).reshape((irrep_d, irrep_d)).T
    C = lsq(irrep.flatten(1, 2), un).reshape((irrep_d, irrep_d)).T
    coef = lsq((B @ A).flatten(), C.flatten())

    print(obj(A, B, coef))
    
    for j in tqdm(range(iters)):
        # minimize wrt A
        A = lsq(
            t.concat([irrep, 100 * coef * B.T @ irrep], dim=0).flatten(1, 2),
            t.concat([ln, 100 * un], dim=0)
        )
        A = A.reshape((irrep_d, irrep_d)).T
        # minimize wrt B
        B = lsq(
            t.concat([irrep, 100 * coef * irrep @ A.T], dim=0).flatten(1, 2),
            t.concat([rn, 100 * un], dim=0)
        )
        B = B.reshape((irrep_d, irrep_d)).T
        # minimize wrt coef
        coef = lsq(
            einops.einsum(irrep, (B @ A).T, 'n d1 d2, d1 d2 -> n'),
            un,
        )
        if j % 10000 == 0:
            print(obj(A, B, coef))
    return A, B, coef

In [105]:
full_rank_optimize(irrep_lneurons[:,0], irrep_rneurons[:,0], irrep_uneurons[:,0])

[tensor(0.0201), tensor(0.0198), tensor(0.0232)]
tensor(0.0631)


  0%|          | 108/100000 [00:00<01:32, 1075.88it/s]

[tensor(1.9850), tensor(0.2180), tensor(0.9523)]
tensor(3.1553)


 11%|█         | 10541/100000 [00:02<00:22, 4041.78it/s]

[tensor(0.0199), tensor(0.0198), tensor(0.7882)]
tensor(0.8280)


...

[tensor(0.0201), tensor(0.0199), tensor(0.7882)]
tensor(0.8281)


 31%|███       | 30780/100000 [00:11<00:28, 2430.27it/s]

[tensor(0.0202), tensor(0.0199), tensor(0.7882)]
tensor(0.8283)


 34%|███▍      | 34113/100000 [00:13<00:26, 2475.28it/s]


KeyboardInterrupt: 

In [17]:
i = 0
rank1_optimize(irrep_lneurons[:,i], irrep_rneurons[:,i], irrep_uneurons[:,i])

  0%|          | 0/100000 [00:00<?, ?it/s]

  1%|          | 729/100000 [00:00<00:26, 3703.76it/s]

tensor(1.2933)


 11%|█         | 10566/100000 [00:02<00:22, 3967.81it/s]

tensor(1.1125)


...

tensor(1.1125)


 91%|█████████ | 90774/100000 [00:23<00:02, 4039.88it/s]

tensor(1.1125)


100%|██████████| 100000/100000 [00:25<00:00, 3875.45it/s]


(tensor([-5.2729e-04,  1.6641e+00,  6.5790e-04,  4.5260e-04, -2.0633e+00,
          6.0780e-04]),
 tensor([-0.0582, -0.0583, -0.2933,  0.3723,  0.3727, -0.2360]),
 tensor([-0.3472, -0.3504,  0.0561,  0.1339,  0.1368, -0.3709]),
 tensor([1.0233]))

In [60]:
rank1_optimize(t.randn_like(irrep_lneurons[:,i]), t.randn_like(irrep_rneurons[:,i]), t.randn_like(irrep_uneurons[:,i]))

  0%|          | 0/100000 [00:00<?, ?it/s]

  1%|          | 592/100000 [00:00<00:33, 3006.44it/s]

tensor(342.7078)


 10%|█         | 10119/100000 [00:03<00:27, 3260.04it/s]


tensor(338.7628)


KeyboardInterrupt: 

In [26]:
c.norm()

tensor(0.2049)

In [26]:
def objective(x, i, d):
    print('a')
    assert len(x) == 3*d + 1
    a = x[0:d]
    b = x[d:2*d]
    c = x[2*d:3*d]
    coef = x[3*d]
    res = sum(
        [
            (irrep_lneurons[:,i] - einops.einsum(b, irrep, a, 'd1, n d1 d2, d2 -> n')).norm()**2,
            (irrep_rneurons[:,i] - einops.einsum(a, irrep, c, 'd1, n d1 d2, d2 -> n')).norm()**2,
            (irrep_uneurons[:,i] - coef * (b*c).sum() * einops.einsum(b, irrep, c, 'd1, n d1 d2, d2 -> n')).norm()**2,
        ]
    )
    print(res.item())
    return res

def norm_a(x, d):
    print('b')
    return (x[:d]**2).sum() - 1

def norm_b(x, d):
    print('c')
    return (x[d:2*d]**2).sum() - 1

def norm_c(x, d):
    print('d')
    return (x[2*d:3*d]**2).sum() - 1

In [27]:
irrep_lneurons[:,i].norm()**2

tensor(89.8718)

In [37]:
from functools import partial
from scipy.optimize import shgo
i = 0
obj = partial(objective, i=i, d=irrep_d)
cons_a = partial(norm_a, d=irrep_d)
cons_b = partial(norm_b, d=irrep_d)
cons_c = partial(norm_c, d=irrep_d)
bounds = [(-10, 10) for _ in range(3*irrep_d + 1)]
res = shgo(obj, bounds, options=dict(f_min=1.), n=150, sampling_method='sobol', )# constraints=[{'type': 'eq', 'fun': cons_a}, {'type': 'eq', 'fun': cons_b}, {'type': 'eq', 'fun': cons_c}], options=dict(f_min=1000.))

In [29]:
res

 message: Optimization terminated successfully.
 success: True
     fun: 180.79434127211326
    funl: [ 1.808e+02]
       x: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
      xl: [[ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]]
     nit: 1
    nfev: 8207
   nlfev: 14
   nljev: 1
   nlhev: 0

In [31]:
irrep_lneurons[:,i].norm()**2

tensor(89.8718)

In [61]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model('model')
a = m.addVars(irrep_d, name='a')
b = m.addVars(irrep_d, name='b')
c = m.addVars(irrep_d, name='c')
coef = m.addVar(name='coef')

obj = sum(
    [
        sum( (irrep_lneurons[g,i] - sum(b[j] * irrep[g,j,k] * a[k] for j in range(irrep_d) for k in range(irrep_d))**2 for g in range(len(group)))),
        sum( (irrep_rneurons[g,i] - sum(a[j] * irrep[g,j,k] * c[k] for j in range(irrep_d) for k in range(irrep_d))**2 for g in range(len(group)))),
        sum( (irrep_uneurons[g,i] - coef * sum(b[j] * irrep[g,j,k] * c[k] for j in range(irrep_d) for k in range(irrep_d))**2 for g in range(len(group)))),
    ]
)

m.addConstr((a**2).sum() == 1, name='norm_a')
m.addConstr((b**2).sum() == 1, name='norm_b')
m.addConstr((c**2).sum() == 1, name='norm_c')

m.optimize()

ModuleNotFoundError: No module named 'cvxpy'

In [63]:
b.T @ irrep @ a

ValueError: Expressions of dimension greater than 2 are not supported.

In [15]:
import cvxpy as cp
i = 0
a, b, c, coef = cp.Variable(irrep_d), cp.Variable(irrep_d), cp.Variable(irrep_d), cp.Variable()

obj = sum(
    [
        sum( (irrep_lneurons[g,i] - sum(b[j] * irrep[g,j,k] * a[k] for j in range(irrep_d) for k in range(irrep_d))**2 for g in range(len(group)))),
        sum( (irrep_rneurons[g,i] - sum(a[j] * irrep[g,j,k] * c[k] for j in range(irrep_d) for k in range(irrep_d))**2 for g in range(len(group)))),
        sum( (irrep_uneurons[g,i] - coef * sum(b[j] * irrep[g,j,k] * c[k] for j in range(irrep_d) for k in range(irrep_d))**2 for g in range(len(group)))),
    ]
)

prob = cp.Problem(cp.Minimize(obj), [cp.norm(a) == 1, cp.norm(b) == 1, cp.norm(c) == 1])
prob.solve(solver=cp.CPLEX)

/home/wilson/.cache/pypoetry/virtualenvs/group_addition-BDyFuYvs-py3.10/lib/python3.10/site-packages/cvxpy/problems/problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
var6[0] @ 1.0 @ var5[0]
var6[0] @ -4.470348358154297e-08 @ var5[1]
var6[0] @ 2.2351741790771484e-08 @ var5[2]
var6[0] @ 0.0 @ var5[3]
var6[1] @ 1.4901161193847656e-08 @ var5[0]
var6[1] @ 1.000000238418579 @ var5[1]
var6[1] @ -7.450580596923828e-08 @ var5[2]
var6[1] @ 2.9802322387695312e-08 @ var5[3]
var6[2] @ 7.450580596923828e-09 @ var5[0]
var6[2] @ -3.725290298461914e-08 @ var5[1]
var6[2] @ 1.0000001192092896 @ var5[2]
var6[2] @ 5.21540641784668e-08 @ var5[3]
var6[3] @ -4.470348358154297e-08 @ var5[0]
var6[3] @ -7.450580596923828e-09 @ var5[1]
var6[3] @ -1.4901161193847656e-08 @ var5[2]
var6[3] @ 1.0 @ var5[3]
var6[0] @ -0.7660841941833496 @ var5[0]
var6[0] @ -0.2618772089481354 @ var5[1]
var6[0] @ 0.5268442630767822 @ var5[2]
var6[0] @ -0.2587868273258209 @ var5[3]
var6[1] @ -0.1273839771747589 @ var5[0]
var6[1] @ 0.2733299434185028 @ var5[1]
var6[1] @ 0.3801375

In [66]:
prob.value